Перебрати всіх дітей і порівняти з попереднім результатом, щоб знайти нових

In [2]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import json
import tqdm
import time


In [4]:
'''
    find last li in pagination. Neet to known the last page in childlist
'''
url = "https://www.msp.gov.ua/children/search.php?form=*&male=*&age_from=*&age_to=*&region=*&brothers=*&needs=*&page=1"
response = requests.get(url)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")
    paragraphs = soup.find_all("ul", class_="pagination") #find pagination block
    for i, paragraph in enumerate(paragraphs, start=1):
        li_elements = paragraph.find_all("li") #find all li in pagination
        
        if li_elements:
            last_li = li_elements[-1] #find al li in pagination block
            last_a = last_li.text.strip() #get text last li
            last_page = int(re.sub(r'\D','',last_a)) #leve only page number    
else:
    print(f"ERROR {response.status_code}")


In [8]:
'''
    This script is time-consuming. Therefore, it should only be run when you want to update data for all children.
    
    get array of all children on page
'''
all_children = []
for i in tqdm.tqdm(range(last_page)): #(last_page):
    url = "https://www.msp.gov.ua/children/search.php?form=*&male=*&age_from=*&age_to=*&region=*&brothers=*&needs=*&page="+str(i+1)
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        child_block = soup.find_all("div",class_="child__item")
        for i, child in enumerate(child_block, start=1):
            link = child.find("a")["href"]
            id_child = re.search(r'n=(\d+)', link)
            if id_child:
                id_child = id_child.group(1)
            else:
                print(f"В картці дитини на сторінці {url}, не знайдено id code")
            #print(id_child,"\n-------------------")
            all_children.append(id_child)
    else:
        print(f"ERROR {response.status_code}")

json_output = json.dumps(all_children, indent=4)
#print(json_output)

with open('all_children.json', 'r') as json_file:
    data = json.load(json_file)

differences = list(set(all_children) ^ set(data))
print(differences)

with open('all_children.json', 'w') as json_file:
    json_file.write(json_output)  # Записуємо JSON в файл

if len(differences)>0:
    with open('differences.json', 'w') as json_file:
        json_file.write(json.dumps(differences, indent=4))  # Записуємо JSON в файл


100%|██████████████████████████████████████████████████████████████████████████████| 1193/1193 [08:20<00:00,  2.39it/s]

['190686', '229384', '459563', '321911', '313760', '296681', '321543', '267266', '392947', '190871', '219449', '373191', '288879', '265434', '303507', '334198', '149229', '418533', '502895', '289753', '430524', '251297', '399891', '329327', '458117', '146023', '568347', '232438', '230905', '388434', '219438', '263856', '239130', '256254', '284452', '379016', '264860', '252053', '407782', '438852', '352151', '241430', '463836', '393863', '277352', '185896', '338594', '458834', '452787', '326463', '165316', '315607', '267243', '340005', '283418', '328230', '454259', '457988', '361810', '232672', '241744', '332429', '440819', '189684', '431016', '371035', '438122', '204162', '227195', '171489', '188817', '275188', '126916', '247826', '290272', '324297', '466280', '370993', '351243', '345172', '402908', '514820', '185506', '449178', '160669', '332604', '373128', '328542', '238316', '196816', '296891', '285141', '256832', '371987', '374158', '392032', '404064', '335104', '63766', '294991', 